In [1]:
debug = True

In [2]:
import termextract.janome
import termextract.core
import collections
from janome.tokenizer import Tokenizer
from pprint import pprint # このサンプルでの処理結果の整形表示のため

#上位c個の重要複合語のリストを返します.c=-1(デフォルト)ですべての複合語を返します.
def Extract_ImportantWords(text,c=-1):
    #ファイル読み込みから、テキストファイル（元の文章から抽出に変更）
    """
    f = open(filename, "r")
    text = f.read()
    f.close
    if(debug):print(text)
    """
    
    t = Tokenizer()
    tokenize_text = t.tokenize(text)
    
    #頻度ベクトル生成
    frequency = termextract.janome.cmp_noun_dict(tokenize_text) #複合語抽出
    
    #LR(ボトムアップ構文解析)のスコアの生成
    lr = termextract.core.score_lr(
        frequency,
        ignore_words=termextract.mecab.IGNORE_WORDS,
        lr_mode=1, average_rate=1)
    #頻度lRスコアの生成(すなわち重要度)
    term_imp = termextract.core.term_importance(frequency, lr)

    #降順に表示
    imp_words = []
    data_collection = collections.Counter(term_imp)
    if(c == -1):
        for cmp_noun, value in data_collection.most_common() :
            imp_words.append(termextract.core.modify_agglutinative_lang(cmp_noun))
    else:
        tmp = 1
        for cmp_noun, value in data_collection.most_common():
            if(tmp > c): break
            imp_words.append(termextract.core.modify_agglutinative_lang(cmp_noun))
            tmp += 1
    if(debug):print(imp_words)
    return imp_words

In [3]:
import requests
from bs4 import BeautifulSoup

# User-Agent

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/83.0.4103.116 Safari/537.36 Edg/83.0.478.64"

def get_html(url, params=None, headers=None):
    """ get_html
    url: データを取得するサイトのURL
    [params]: 検索サイトのパラメーター {x: param}
    [headers]: カスタムヘッダー情報
    """
    try:
        # データ取得
        resp = requests.get(url, params=params, headers=headers)
        resp.encoding = 'utf-8'
        # 要素の抽出
        soup = BeautifulSoup(resp.text, "html.parser")
        return soup
    except Exception as e:
        return None

def get_search_url(word):
    """ get_search_url
    word: 検索するワード
    """
    try:
        # google 検索
        search_url = "https://www.google.co.jp/search"
        search_params = {"q": word}
        search_headers = {"User-Agent": user_agent}
        # データ取得
        soup = get_html(search_url, search_params, search_headers)
        if soup != None:
            #class = r <a href>
            tags = soup.select(".r > a")
            urls = [tag.get("href") for tag in tags]
            return urls
        else:
                return None
    except Exception as e:
        return None

In [4]:
def extract_textFhtml(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    text=soup.get_text()
    lines=[]
    for line in text.splitlines():
        lines.append(line.strip())
    text="".join(line for line in lines if line)
    return text

In [5]:
def CreateAdditionalExplanation(filename,c=-1):
    additionals = []
    importances = Extract_ImportantWords(filename,c)
    for word in importances:
        try:
            urls = get_search_url(word)
            if(urls[0] != None):
                soup = get_html(urls[0])
                text = extract_textFhtml(soup)
                additionals.append([word,text])
        except Exception as e:
            if(debug):print("エラー")
    return additionals

In [7]:
print(CreateAdditionalExplanation("out.txt",1))

[['経路表', 'インターネット10分講座：経路制御 - JPNICJavaScriptが無効になっています。JavaScriptを有効にしてご利用いただくことで、より快適にご利用いただけます。メインコンテンツへジャンプするホームQ&AサイトマップアクセスENJPNICはインターネットの円滑な運営を支えるための組織ですWHOIS 検索サイト内検索WHOISとは？｜JPNIC WHOIS GatewayGoogleBingJPNICJPNICとはIPアドレスインターネットの基礎ドメイン名インターネットの歴史・統計トピックスとお知らせインターネットの技術ライブラリインターネットガバナンスICANN情報Web更新履歴Q&AイベントカレンダーWHOISIPアドレス・AS番号指定事業者向け各種申請手続/お知らせIPアドレスAS番号特殊用途用PIアドレス歴史的PIアドレスJPIRR指定事業者向けページIPアドレストピックスドキュメント一覧一般ユーザーの方IPアドレスの基本情報IPアドレスが欲しい時にはIPv6関連JPIPRJPNIC会員会員向け情報/各種変更手続会員ページトップメンバーズラウンジ会員ロゴ情報変更(正会員)情報変更(賛助会員)情報変更(個人会員)入会のご案内会員の種類・会費・メリット入会手続情報よくある質問JPNIC会員リストWHOIS検索サイト内検索用語集ページトップへ検索するGoogleBingよく検索されるキーワードIPv6DNSHTTPWHOISBGPIPv4ドメイン名とはIPアドレスFQDNインターネットWHOIS検索サイト内検索WHOISとは？｜JPNIC WHOIS GatewayGoogleBingよく検索されるキーワードIPv6DNSHTTPWHOISBGPIPv4ドメイン名とはIPアドレスFQDNインターネットJPNICJPNICJPNICとはIPアドレスインターネットの基礎ドメイン名インターネットガバナンスインターネットの技術インターネットの歴史・統計ライブラリトピックスとお知らせWeb更新履歴Q&AイベントカレンダーWHOISIPアドレス・AS番号IPアドレス・AS番号IPアドレスAS番号特殊用途用PIアドレス歴史的PIアドレスJPIPRIPアドレストピックスドキュメント一覧IPアドレスの基本情報IPアドレスが欲しい時にはIPv6関連JP